#Decision Support System
This notebook enables researchers to retrieve citations through accessing multiple API's at once.


**Requirements:**


*   API keys for Pubmed and Elsevier API
*   For Elsevier's API, you must have a connection via an Academic IP address (Can be through VPN)

**We have several options:**
1.   Automated data retrieval, manual labeling and automated study selection
2.   Manual labeling and automated study selection 
3.   Automated study selection

To execute option 2 and 3, please upload your database results to the system, and run the corresponding cells.






In [ ]:
#@title Run this cell to install to import all required libraries { display-mode: "form" }
!pip install elsapy
!pip install biopython
from Bio import Entrez
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import pandas as pd
import ipywidgets as widgets
import csv 
from Bio import Medline
import requests
import re
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
import numpy as np

# Log in to google drive to access files
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/'My Drive'/'Decision Support System'

# Custom scripts
from metrics import WorkSavedOverSamplingAtRecall
from preprocessing import Preprocessor
from model import multichannel_cnn


In [ ]:
#@title # Step 1 - Data Retrieval { display-mode: "form" }
#@markdown ---

#@markdown ### General information
elsevier_api_key = '' #@param {type:'string'} 
springer_api_key = '' #@param {type:'string'} 
email = "lorem@ipsum.org" #@param {type:'string'}
#lorem@ipsum.org
Entrez.email = email
#@markdown ---

#@markdown ### Databases
pubmed = True #@param {type:"boolean"}
sciencedirect = False #@param {type:"boolean"}
springer = True #@param {type:"boolean"}

#@markdown ---

#@markdown ### Search query
search_query = "\"Systematic review\" AND automation" #@param {type:'string'}
#(Automation OR Automate OR Automates OR Automating) AND (\"Systematic Literature Review\" OR \"Systematic Review\")

#@markdown ### Search fields
## @markdown If you didn't choose the database to search, it does not matter what field you choose for that particular database

# EXAMPLE: Here, we request all possible fields for the PubMed database.
# print possible_fields_df if you want to use another field that Title/Abstract
# Through Entrez.einfo, you can also find all other databases and their fields.
# record = Entrez.read(Entrez.einfo(db='pubmed'))
# possible_fields_df = pd.DataFrame(record['DbInfo']['FieldList'])

pubmed_field = 'TIAB' #@param ['ALL', 'UID', 'FILT', 'TITL', 'WORD', 'MESH', 'MAJR', 'AUTH', 'JOUR', 'AFFL', 'ECNO', 'SUBS', 'PDAT', 'EDAT', 'VOL', 'PAGE', 'PTYP', 'LANG', 'ISS', 'SUBH', 'SI', 'MHDA', 'TIAB', 'OTRM', 'INVR', 'COLN', 'CNTY', 'PAPX', 'GRNT', 'MDAT', 'CDAT', 'PID', 'FAUT', 'FULL', 'FINV', 'TT', 'LAUT', 'PPDT', 'EPDT', 'LID', 'CRDT', 'BOOK', 'ED', 'ISBN', 'PUBN', 'AUCL', 'EID', 'DSO', 'AUID', 'PS', 'COIS'] {type:"string"}
springer_field = 'all' #@param ['all', 'title', 'orgname', 'journal', 'book', 'name'] {type:"string"}

#@markdown ### Max returned citations per database
pubmed_maxret = 200 #@param {type:"number"}
springer_maxret = 300 #@param {type:"number"}

#@markdown ### Timeframe
##@markdown Springer only searches on year, not exact date
start_date = '2000/01/01' #@param {type:"date"}
end_date = '2021/01/01' #@param {type:"date"}

#@markdown ---

#@markdown Press SHIFT+ENTER or click the play button to execute the query

# The code below descibes the retrieval of documents from the API's

def springer_query_tagging(query, tag):
    '''
    This function ensures that the search query for Springer is correctly tagged
    '''
    if tag == 'all':
        # query does not need to be tagged
        return query

    # split the query, while keeping all characters
    query_lst = (re.split('(\W)', query))

    tag = tag + ':'
    boolean_operators = ['OR', 'AND', 'NOT']
    brackets = ['(', ')']    
    spaces = [' ', '']        
    iseven = False

    tagged_query_lst = []

    # go through every item in the query, if it is a term, add the tag before it
    for item in query_lst:
        if item.upper() not in boolean_operators:
            if item not in brackets and item not in spaces: 
                if item is not '"' and not iseven:
                    tagged_query_lst.append(tag+item)
                elif item is '"' and not iseven:
                    tagged_query_lst.append(tag+item)
                    iseven = not iseven
                elif item is '"' and iseven:
                    tagged_query_lst.append(item)
                    iseven = not iseven
                else:
                    tagged_query_lst.append(item)
            elif item in brackets:
                pass
            else:
                tagged_query_lst.append(item)
        else:
            tagged_query_lst.append(item)
    return ''.join(tagged_query_lst)

def springer_search(q, api_key, start_date, end_date, retmax):
    # adopt a retry strategy for http requests
    retry_strategy = Retry(
                    total=3,
                    status_forcelist=[429, 500, 502, 503, 504],
                    method_whitelist=["HEAD", "GET", "OPTIONS"]
                    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)
    http.mount("http://", adapter)

    start_result_position=1
    sort = 'sequence'
    results = []

    # Get the query results untill the maximum number of documents has been retrieved, or no documents are available anymore 
    while True:
        x = http.get(f'http://api.springernature.com/meta/v2/json?q={q} sort:{sort}&api_key={api_key}&s={start_result_position}&p={retmax}&date-facet-mode=between&facet-start-year={start_date[:4]}&showAll=true&facet-end-year={end_date[:4]}')
        start_result_position += 100
        results.append(pd.DataFrame(x.json()['records']))

        if int(x.json()['result'][0]['recordsDisplayed']) == 0 or start_result_position > retmax:
            return pd.concat(results), x.json()['query']

    return pd.concat(results), x.json()['query']

query_results = []

if pubmed:
    print('--- Pubmed ---')
    # Find all article IDs containing search query, sorted by relevance
    handle = Entrez.esearch(db="pubmed", retmax=pubmed_maxret, term=search_query, sorted='relevance', idtype="acc", field=pubmed_field, mindate=start_date, maxdate=end_date)
    record = Entrez.read(handle)
    handle.close()

    # Retrieve all article data by ID
    idlist = record["IdList"]
    handle = Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode="text")
    docs = []

    # Parse data in medline format and save to file
    articles = Medline.parse(handle)
    for article in articles:
        docs.append(article)
    pubmed_df = pd.DataFrame(docs)
    pubmed_df = pubmed_df.rename(columns={'TI': 'title', 'AB':'abstract', 'AID': 'identifier', 'DP':'publicationDate'})
    pubmed_df['database'] = 'pubmed'

    query_results.append(pubmed_df)
    print("The query interpreted by PubMed: {} \nThis query resulted into {} records found.".format(record['QueryTranslation'], record["Count"]))


if springer:
    print('--- Springer ---')
    # Springer allows a maximum of 100 returns
    tagged_query = springer_query_tagging(search_query, springer_field)
    springer_df, springer_query = springer_search(tagged_query, springer_api_key, start_date, end_date, retmax=springer_maxret)
    springer_df['database'] = 'springer'
    query_results.append(springer_df)
    print("The tagged query: {} \nThis query resulted into {} records found.".format(springer_query, len(springer_df)))


if sciencedirect:
    print('--- ScienceDirect ---')
    ## Initialize client
    client = ElsClient(elsevier_api_key)

    # If get_all = True, then all results will be retrieved, with a maximum of 5000, 
    # otherwise, 20 results will be retrieved (1 API request)
    get_all = False

    ## Initialize doc search object using ScienceDirect and execute search, 
    #   retrieving all results
    doc_srch = ElsSearch(search_query, 'sciencedirect')
    doc_srch.execute(client, get_all=get_all)
    print("doc_srch has", doc_srch.len_res(), "results.")

    sciencedirect_df = doc_srch.results_df
    query_results.append(sciencedirect_df)

if len(query_results) == 0:
    print('No database selected')
else:
    with pd.ExcelWriter('datasets/query_results.xlsx') as writer:  
        # Write each dataframe to a different worksheet.
        for result in query_results:
            result.to_excel(writer, sheet_name=result['database'].iloc[0])

    print('Articles have been saved to datasets/query_results.xlsx')

In [ ]:
#@title # Dataset splitting
#@markdown This cell should be executed before executing Step 2 or 3.

#@markdown ---
#@markdown Put your data file name below.
slr_excel_file = 'query_results.xlsx' #@param {type:"string"}
#@markdown Make sure each database has its own results sheet, containing at least the following columns: 'title', 'publicationDate', 'identifier', 'database', and 'abstract'

query_results_file = pd.ExcelFile('datasets/'+slr_excel_file)
sheets = query_results_file.sheet_names
Xtrains = []
Xtests = []
ytrains = []
ytests = []
for sheet in sheets:
    query_results = query_results_file.parse(sheet)
    if 'label' not in query_results.columns:
        query_results['label'] = np.nan
    
    # We want to split each databases' results independently, as they have been sorted based on relevance by the databases 
    Xtrain, Xtest, ytrain, ytest = train_test_split(query_results[['title', 'publicationDate', 'identifier', 'abstract', 'database']], query_results['label'], test_size=0.5, shuffle=False)
    Xtrains.append(Xtrain)
    Xtests.append(Xtest)
    ytrains.append(ytrain)
    ytests.append(ytest)

# We want to concatenate the search results from the databases, and alternating the rows to keep the most relevant articles on top for the training set.
Xtrain = pd.concat(Xtrains).sort_index(kind='merge')
ytrain = pd.concat(ytrains).sort_index(kind='merge')
Xtest = pd.concat(Xtests)
ytest = pd.concat(ytests)
train = pd.concat([Xtrain, ytrain], axis=1)
test = pd.concat([Xtest, ytest], axis=1)
train.to_excel('datasets/train.xlsx')
test.to_excel('datasets/test.xlsx')

print('Train and test set have been split.')

In [ ]:
#@title # Step 2 - Manual labeling
#@markdown ---


#@markdown Run this cell to start manual classification of the train set
train_set = pd.read_excel('datasets/train.xlsx')
train_set = train_set[['title', 'publicationDate', 'identifier', 'abstract', 'database', 'label']]


already_classified = train_set[~train_set['label'].isna()]
train = train_set[train_set['label'].isna()].reset_index()


i = 0

def on_button_clicked_include(b):
    global i
    train['label'][i] = True
    clear_output()
    if i < len(train):
        i += 1
        display(hbox)
        printable_abs = re.sub("(.{150} )", "\\1\n", train['abstract'][i], 0, re.DOTALL)
        print('\nReview {} out of {}.\n\nTitle: {}\nYear: {}\nIdentifier: {}\nDatabase:{}\nAbstract: {}'.format(i+1, len(train)+1, train['title'][i], train['publicationDate'][i], train['identifier'][i], train['database'][i], printable_abs))
    else:
        print('All citations have been reviewed.')
        save_set = pd.concat([already_classified, train], ignore_index=True)
        save_set.to_excel('datasets/train.xlsx')
        print('Training set updated and saved.')

def on_button_clicked_exclude(b):
    global i
    train['label'][i] = False
    clear_output()
    if i < len(train):
        i += 1
        display(hbox)
        printable_abs = re.sub("(.{150} )", "\\1\n", train['abstract'][i], 0, re.DOTALL)
        print('\nReview {} out of {}.\n\nTitle: {}\nYear: {}\nIdentifier: {}\nDatabase:{}\nAbstract: {}'.format(i+1, len(train)+1, train['title'][i], train['publicationDate'][i], train['identifier'][i], train['database'][i], printable_abs))
    else:
        print('All citations have been reviewed.')
        save_set = pd.concat([already_classified, train], ignore_index=True)
        save_set.to_excel('datasets/train.xlsx')
        print('Training set updated and saved.')

def on_button_clicked_save_and_exit(b):
    save_set = pd.concat([already_classified, train], ignore_index=True)
    save_set.to_excel('datasets/train.xlsx')
    print('Training set updated and saved.')

if len(train) > 0:
    # Show buttons
    include = widgets.Button(description="Include")
    exclude = widgets.Button(description="Exclude")
    save_and_exit = widgets.Button(description="Save and Exit")
    hbox = widgets.HBox([include, exclude, save_and_exit])
    display(hbox)

    include.on_click(on_button_clicked_include)
    exclude.on_click(on_button_clicked_exclude)
    save_and_exit.on_click(on_button_clicked_save_and_exit)

    printable_abs = re.sub("(.{150} )", "\\1\n", train['abstract'][i], 0, re.DOTALL)
    print('\nReview {} out of {}.\n\nTitle: {}\nYear: {}\nIdentifier: {}\nDatabase:{}\nAbstract: {}'.format(i+1, len(train)+1, train['title'][i], train['publicationDate'][i], train['identifier'][i], train['database'][i], printable_abs))

else:
    print('All articles have been labeled.')

In [ ]:
#@title # Step 3 - Citation Screening
train_path = 'datasets/train.xlsx'
test_path = 'datasets/test.xlsx'
embedding_path = 'embeddings/glove.6B.100d.txt'
buffer_size = 512
batch_size = 100
epochs=15

preprocessor = Preprocessor()
train_ds, test_ds, steps_per_epoch = preprocessor.get_tf_datasets(train_path, test_path, embedding_path, batch_size, batch_size, show_imbalance=True)

max_len = preprocessor.get_max_sequence_length()
model = multichannel_cnn()

print('Training the model, this may take a few minutes depending on your dataset size')
history = model.fit(train_ds, steps_per_epoch=steps_per_epoch, epochs=epochs, verbose=0)

# Make predictions and sort them in descending order to an excel file.
test = preprocessor._test
test['prediction'] = model.predict(test_ds) 
test = test.sort_values('prediction', ascending=False)
test.to_excel('datasets/predictions.xlsx')
print('Predictions have been saved in datasets/predictions.xlsx')

In [ ]:
#@title # Model evaluation
#@markdown This cell can be executed to assess the performance of our model. This can only be performed when test dataset has also been labeled.
evaluation = model.evaluate(test_ds, verbose=0, return_dict=True)

test = preprocessor._test
test = test.sort_values('prediction', ascending=True)
len_before = len(test)
test = test.loc[test[(test != 0).all(axis=1)].first_valid_index():]
len_after = len(test)

print('Saved reading {} articles. A WSS@95% score of {:.2f}% was achieved.'.format(len_before-len_after, evaluation['wss_95']*100))